# Tarea 5 - Árbol de Desición

***Inteligencia Artificial***

***II Semestre 2024***

***Tecnológico de Costa Rica***

Estudiantes: 
- Esteban Guzmán
- Rolando Mora

In [2]:
import pandas as pd
import numpy as np